## Pivoting your data

In this chapter, you will go one step further in generating personalized recommendations — you will find items that users, similar to the one you are making recommendations for, have liked.

The first step you will need to start with is formatting your data. You begin with a dataset containing users and their ratings as individual rows with the following columns:

    user: User ID
    title: Title of the movie
    rating: Rating the user gave the movie

You will need to transform the DataFrame into a user rating matrix where each row represents a user, and each column represents the movies on the platform. This will allow you to easily compare users and their preferences.

### Instructions 1/3
    - Inspect the first five rows of the user_ratings DataFrame to observe which columns would be most appropriate to pivot the data around.

In [ ]:
# Inspect the first 5 rows of user_ratings
print(user_ratings.head())

### Instructions 3/3
    - Transform the user_ratings DataFrame to a DataFrame containing ratings with one row per user and one column per movie and call it user_ratings_table

In [ ]:
# Transform the table
user_ratings_table = user_ratings.pivot(index='userId', columns='title', values='rating')
# Inspect the transformed table
print(user_ratings_table.head())

## Challenges with missing values

You may have noticed that the pivoted DataFrames you have been working with often have missing data. This is to be expected since users rarely see all movies, and most movies are not seen by everyone, resulting in gaps in the user-rating matrix.

In this exercise, you will explore another subset of the user ratings table user_ratings_subset that has missing values and observe how different approaches in dealing with missing data may impact its usability.

### Instructions 2/3
    - Fill the gaps in the user_ratings_subset with zeros.
    - Print and inspect the results.

In [ ]:
# Fill in missing values with 0
user_ratings_table_filled = user_ratings_subset.fillna(0)

# Inspect the result
print(user_ratings_table_filled)

## Compensating for incomplete data

For most datasets, the majority of users will have rated only a small number of items. As you saw in the last exercise, how you deal with users who do not have ratings for an item can greatly influence the validity of your models.

In this exercise, you will fill in missing data with information that should not bias the data that you do have.

You'll get the average score each user has given across all their ratings, and then use this average to center the users' scores around zero. Finally, you'll be able to fill in the empty values with zeros, which is now a neutral score, minimizing the impact on their overall profile, but still allowing the comparison of users.

user_ratings_table with a row per user has been loaded for you.

### Instructions
    - Find the average of the ratings given by each user in user_ratings_table and store them as avg_ratings.
    - Subtract the row averages from each row in user_ratings_table, and store it as user_ratings_table_centered.
    - Fill the empty values in the newly created user_ratings_table_centered with zeros.

In [ ]:
# Get the average rating for each user 
avg_ratings = user_ratings_table.mean(axis=1)

# Center each users ratings around 0
user_ratings_table_centered = user_ratings_table.sub(avg_ratings, axis=0)

# Fill in the missing data with 0s
user_ratings_table_normed = user_ratings_table_centered.fillna(0)

## User-based to item-based

By now you have a dataset with no empty values that is primed for use.

In the preceding video, you learned about both user-based recommendations and item-based recommendations. User-based recommendations compare amongst users, and item-based recommendations compare different items.

In other words, you could use user-based data to find similar users based on how they rated different movies, while you could use item-based data to find similar movies based on how they have been rated by the users.

In this exercise, you will switch between the two and compare their values.

user_ratings_subset, a subset of the user-based DataFrame you have been working with, has been loaded for you.

### Instructions 2/3
    - Transpose the user_ratings_subset table so that it is indexed by the movies and store the result as movie_ratings_subset.

In [ ]:
# Transpose the user_ratings_subset DataFrame
movie_ratings_subset = user_ratings_subset.T

print(movie_ratings_subset)

## Similar and different movie ratings

Some types of movies might be liked by one group of people, but hated by another. This might reflect the type of movie far more than its quality. Take, for example, horror movies — many people absolutely love them, while others hate them.

By understanding which movies were reviewed in a similar way, we can often find very similar movies.

In this exercise, you will compare movies and see whether they have received similar reviewing patterns.

The DataFrame movie_ratings_centered has been loaded with a row per movie, and the centered ratings it received as the values.

### Instructions 1/2
    - Assign the values for Star Wars: Episode IV and Star Wars: Episode V to sw_IV and sw_V.
    - Find their cosine similarity.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Assign the arrays to variables
sw_IV = movie_ratings_centered.loc['Star Wars: Episode IV - A New Hope (1977)', :].values.reshape(1, -1)
sw_V = movie_ratings_centered.loc['Star Wars: Episode V - The Empire Strikes Back (1980)', :].values.reshape(1, -1)

# Find the similarity between two Star Wars movies
similarity_A = cosine_similarity(sw_IV, sw_V)
print(similarity_A)

### Instructions 2/2
    - Find the cosine similarity between the ratings for Jurassic Park (jurassic_park) and Pulp Fiction (pulp_fiction).

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Assign the arrays to variables
sw_IV = movie_ratings_centered.loc['Star Wars: Episode IV - A New Hope (1977)', :].values.reshape(1, -1)
sw_V = movie_ratings_centered.loc['Star Wars: Episode V - The Empire Strikes Back (1980)', :].values.reshape(1, -1)

# Find the similarity between two Star Wars movies
similarity_A = cosine_similarity(sw_IV, sw_V)

# Assign the arrays to variables
jurassic_park = movie_ratings_centered.loc['Jurassic Park (1993)', :].values.reshape(1, -1)
pulp_fiction = movie_ratings_centered.loc['Pulp Fiction (1994)', :].values.reshape(1, -1)

# Find the similarity between Pulp Fiction and Jurassic Park
similarity_B = cosine_similarity(jurassic_park, pulp_fiction)
print(similarity_B)

## Finding similarly liked movies

Just like you calculated the similarity between two movies, you can calculate it across all users to find the most similar movie to another based on how users have rated them.

The approach is similar to how you worked with content-based filtering.

You will find the similarity scores between all movies and then drill down on the movie of interest by isolating and sorting the column containing its similarity scores.

movie_ratings_centered has once again been loaded, containing each movie as a row, and their centered ratings stored as the values.

### Instructions
    - Calculate the similarity matrix between all movies in movie_ratings_centered and store it as similarities.
    - Wrap the similarities matrix in a DataFrame, with the indices of movie_ratings_centered as the columns and rows.